<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/modal_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio app: Dev in Notebook, Deployment to Modal

This notebook shows how to develop a Gradio app in a notebook.
The app then can be automatically deployed to Modal.com.
- Export Notebook to .py-file automatically (via nbdev)
- Serve or deploy .py-file to Modal.com automatically

Code blocks wich are needed for the final .py-file (for Modal.com) are marked with `#|export`

Sources:  
- https://modal.com/docs/guide/ex/vision_model_training
- https://github.com/modal-labs/modal-examples/blob/main/06_gpu_and_ml/vision_model_training.py


In [ ]:
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])

In [ ]:
!pip install modal nbdev fastapi gradio -q

In [ ]:
import json
import os
from getpass import getpass
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [ ]:
import os
os.environ["MODAL_TOKEN_ID"] = CREDS['MODAL']['MODAL_TOKEN_ID']['credential']
os.environ["MODAL_TOKEN_SECRET"] = CREDS['MODAL']['MODAL_TOKEN_SECRET']['credential']

# Part 1: Modal App (init)



In [ ]:
#|export

# Modal app

from modal import (
    Stub,
    Image,
    asgi_app
)
from fastapi import FastAPI
import gradio as gr
from gradio.routes import mount_gradio_app

stub = Stub(name="gradio-modal")

image = Image.debian_slim().pip_install(
    "gradio~=3.6"
)

# Part 2: The Gradio App (works in Notebook and in Modal)

In [ ]:
#|export

# Gradio app

def greet(name):
    return "Hello " + name + "!!!"

# variant 1: gr.Interface
demo = gr.Interface(
    fn=greet,
    inputs="text",
    outputs="text"
)

# variant 2: gr.Blocks
with gr.Blocks() as demo:
    name = gr.Textbox(label="Name")
    output = gr.Textbox(label="Output Box")
    greet_btn = gr.Button("Greet")
    greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

In [ ]:
# Start Gradio App in Notebook
demo.launch(quiet=True, share=False, debug=True)

# Part 3: Modal app (main)

In [ ]:
#|export

# Modal app

web_app = FastAPI()

@stub.function(image=image)
@asgi_app()
def fastapi_app():

    return mount_gradio_app(
        app=web_app,
        blocks=demo,
        path="/",
    )

# Export production file for Modal with nbdev

The notebook file will be copied from Google drive to the current notebook environment.

Alternative?: %notebook modal_hello_world2.ipynb

In [ ]:
# Connect Google Drive
# This and the following cell can be skipped it the notebook file is copied manually to the current notebook environment
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Copy Notebook to local path
!cp "/content/drive/MyDrive/Colab Notebooks/modal-gradio.ipynb" /content/

In [ ]:
# Export source code marked with #|export
from nbdev.export import nb_export
nb_export('/content/modal-gradio.ipynb', lib_path='.', name='gradio_modal')

# Run in Modal

In [ ]:
# dev server
!modal serve gradio_modal.py

In [ ]:
# Deploy server permanently
!modal deploy gradio_modal.py

# Idea: Modify exported code if necessary

In [ ]:
# Optimize exported source code
# Creates a backup file automatically
import io
import os
from datetime import datetime

# load
f= open('example.py','r')
if f.mode == 'r':
      source_code =f.read()
      #print(source_code)
f.close()

# modifiy code
source_code = source_code.replace("#stub", "stub")
source_code = source_code.replace("#@stub", "@stub")
source_code = source_code + "\nTest"

# save
current_date_time = datetime.now().strftime("_%Y-%m-%d_%H-%M-%S")
os.rename('example.py', 'example' + current_date_time + '.py')
f= open('example.py','w+')
f.write(source_code)
f.close()